In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


a=pd.read_csv(r"/Users/vijay/Downloads/titanic (1)/train.csv")
a['Sex'].replace(['male','female'],[1,2], inplace=True)
a['Age'].replace([np.NaN],[30], inplace=True)
a['SibSp'].replace([np.NaN],[0], inplace=True)
a['Parch'].replace([np.NaN],[0], inplace=True)
a['Fare'].replace([np.NaN],[0], inplace=True)
a['Pclass'].replace([np.NaN],[2], inplace=True)
a['Embarked'].replace(['S','C','Q'],[1,2,3], inplace=True)
a['Embarked'].replace([np.NaN],[2], inplace=True)
a['Survived'].replace([np.NaN],[1], inplace=True)
a

In [ ]:
y=np.array(a)[:,1]
y=y.astype('int')


In [ ]:
a_reduced=a[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
a_reduced

In [ ]:
data=np.array(a_reduced)
data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score as cvs
from sklearn.model_selection import KFold

kf=KFold(n_splits=10,shuffle=True,random_state=10)
rfc=RandomForestClassifier(random_state=13,max_depth=4)
scores = cvs(rfc,data,y,cv=kf)
print(scores)
print(scores.mean())


In [ ]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(data, y, random_state=20,stratify=y)
rfc=RandomForestClassifier(random_state=14,max_depth=4)
rfc.fit(X_train,y_train)
test_score=rfc.score(X_test,y_test)
train_score=rfc.score(X_train,y_train)
print(train_score)
print(test_score)

In [ ]:
rfc.feature_importances_

In [ ]:
a1=pd.read_csv(r"/Users/vijay/Downloads/titanic (1)/test.csv")
a1['Sex'].replace(['male','female'],[1,2], inplace=True)
a1['Age'].replace([np.NaN],[30], inplace=True)
a1['SibSp'].replace([np.NaN],[0], inplace=True)
a1['Parch'].replace([np.NaN],[0], inplace=True)
a1['Fare'].replace([np.NaN],[0], inplace=True)
a1['Pclass'].replace([np.NaN],[2], inplace=True)
a1['Embarked'].replace(['S','C','Q'],[1,2,3], inplace=True)
a1['Embarked'].replace([np.NaN],[2], inplace=True)
a1

In [ ]:
a1_reduced=a1[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
a1_reduced

In [ ]:
a1_reduced_data=np.array(a1_reduced)
a1_reduced_data

In [ ]:
rfc=RandomForestClassifier(random_state=14,max_depth=7)
rfc.fit(data,y)
prediction=rfc.predict(a1_reduced_data)
prediction

In [ ]:
PassengerId=np.arange(892,1310,1).reshape(-1,1)
PassengerId
result=np.hstack([PassengerId,prediction.reshape(-1,1)])


In [ ]:
dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
df
  

In [ ]:
age_binned=np.digitize(data[:,2],bins=np.array([0,12,18,35,50,70,100]))
fare_binned=np.digitize(data[:,5],bins=np.array([0,15,40,70,100,200,300,400,550]))


In [ ]:
data_binned_a=np.hstack([data[:,0:2],age_binned.reshape(-1,1),data[:,3:5],fare_binned.reshape(-1,1),data[:,6].reshape(-1,1)])

In [ ]:
data_binned_a

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
ohe.fit(data_binned_a)
data_binned_encoded_a=ohe.transform(data_binned_a).toarray()


In [ ]:
data_binned_encoded_a

In [ ]:
data_binned_encoded_a.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(random_state=13,max_iter=10000,penalty='l2',C=1,solver='lbfgs')
scores = cvs(logreg,data,y,cv=kf)
print(scores)
print(scores.mean())

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(random_state=25,max_iter=10000)
mlp.fit(X_train,y_train)
print(mlp.score(X_train,y_train))
print(mlp.score(X_test,y_test))

In [ ]:

X_train, X_test, y_train, y_test = tts(data_binned_encoded_a, y, random_state=20,stratify=y)
mlp=MLPClassifier(random_state=25,max_iter=10000,learning_rate_init=0.001,activation='tanh',solver='adam')
mlp.fit(X_train,y_train)
print(mlp.score(X_train,y_train))
print(mlp.score(X_test,y_test))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(degree=3,interaction_only=True, include_bias=False)
poly_d=poly.fit_transform(data_binned_encoded_a)
poly_d.shape

In [ ]:
X_train, X_test, y_train, y_test = tts(poly_d, y, random_state=20,stratify=y)
mlp=MLPClassifier(random_state=25,max_iter=10000,learning_rate_init=0.001,activation='tanh',solver='adam')
mlp.fit(X_train,y_train)
print(mlp.score(X_train,y_train))
print(mlp.score(X_test,y_test))

In [ ]:
from sklearn.svm import SVC
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                  'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
X_train, X_test, y_train, y_test = tts(data, y, random_state=20,stratify=y)
svc=SVC(random_state=25)
svc.fit(X_train,y_train)
print(svc.score(X_train,y_train))
print(svc.score(X_test,y_test))

In [ ]:
from sklearn.model_selection import GridSearchCV 
grid_search = GridSearchCV(svc, param_grid, cv=10)
grid_search.fit(X_train, y_train)
print(grid_search.score(X_test, y_test))

In [ ]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

In [ ]:
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
age_binned=np.digitize(a1_reduced_data[:,2],bins=np.array([0,16,45,100]))
fare_binned=np.digitize(a1_reduced_data[:,5],bins=np.array([0,15,40,70,100,200,300,400,550]))
data_binned_a1=np.hstack([a1_reduced_data[:,0:2],age_binned.reshape(-1,1),a1_reduced_data[:,3:5],fare_binned.reshape(-1,1),a1_reduced_data[:,6].reshape(-1,1)])


In [ ]:
data_binned_a1

In [ ]:
result=np.hstack([PassengerId,prediction.reshape(-1,1)])

dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
df

In [ ]:
#df.to_csv(r"/Users/vijay/Downloads/titanic (1)/result.csv",index=False)

In [ ]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(data, y, random_state=20,stratify=y)
rfc=RandomForestClassifier(random_state=14,max_depth=4)
rfc.fit(X_train,y_train)
test_score=rfc.score(X_test,y_test)
train_score=rfc.score(X_train,y_train)
print(train_score)
print(test_score)

In [ ]:
prediction=rfc.predict(a1_reduced_data)

In [ ]:
result=np.hstack([PassengerId,prediction.reshape(-1,1)])

dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
df

In [ ]:
#df.to_csv(r"/Users/vijay/Downloads/titanic (1)/result.csv",index=False)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier(random_state=10,max_depth=2,learning_rate=0.2, n_estimators=100,min_samples_split=4,
                              min_samples_leaf=1,max_features=1)
gbc.fit(X_train,y_train)
train_score=gbc.score(X_train,y_train)
test_score=gbc.score(X_test,y_test)
print(train_score)
print(test_score)

In [ ]:
prediction=gbc.predict(a1_reduced_data)
result=np.hstack([PassengerId,prediction.reshape(-1,1)])
dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
#df.to_csv(r"/Users/vijay/Downloads/titanic (1)/result.csv",index=False)

In [ ]:
scores = cvs(gbc,data,y,cv=kf)
print(scores)
print(scores.mean())

In [ ]:
param_grid = {'max_depth': [2,3,4],
              'learning_rate': [0.1,0.2],
              'n_estimators': [ 50, 100, 150],
              'min_samples_split': [3,4,5],
              'min_samples_leaf': [1,2],
              'max_features': [1,2]}
gbc=GradientBoostingClassifier()
grid_search = GridSearchCV(gbc, param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.score(X_test, y_test))

In [ ]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

In [ ]:
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
gbc=GradientBoostingClassifier(max_depth=4, max_features=2, min_samples_split=3,
                           n_estimators=50,learning_rate=0.1)
gbc.fit(X_train, y_train)
prediction=gbc.predict(a1_reduced_data)

In [ ]:
result=np.hstack([PassengerId,prediction.reshape(-1,1)])
dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
#df.to_csv(r"/Users/vijay/Downloads/titanic (1)/result.csv",index=False)

In [ ]:
X_train, X_test, y_train, y_test = tts(data_binned_a, y, random_state=20,stratify=y)
param_grid = {'max_depth': [2,3,4],
              'learning_rate': [0.1,0.2],
              'n_estimators': [ 50, 100, 150],
              'min_samples_split': [3,4,5],
              'min_samples_leaf': [1,2],
              'max_features': [1,2]}
gbc=GradientBoostingClassifier()
grid_search = GridSearchCV(gbc, param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.score(X_test, y_test))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
X_train, X_test, y_train, y_test = tts(data, y, random_state=20,stratify=y)
gbc=GradientBoostingClassifier(random_state=10,max_depth=2,learning_rate=0.2, n_estimators=100,min_samples_split=4,
                              min_samples_leaf=2,max_features=1)
gbc.fit(X_train,y_train)
train_score=gbc.score(X_train,y_train)
test_score=gbc.score(X_test,y_test)
print(train_score)
print(test_score)

In [ ]:
prediction=gbc.predict(a1_reduced_data)
result=np.hstack([PassengerId,prediction.reshape(-1,1)])
dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
df.to_csv(r"/Users/vijay/Downloads/titanic (1)/result.csv",index=False)

In [ ]:
# End of project
# Support vector with some parameter tuning
# and gradient_boost with some parameter tuning gives highest score 

In [ ]:
# But some feature engineering is required like 

#handling missing age values
#prefixes like Master/Mr./Miss/Mrs
#can we extract information from ticket number e.g tickets bought together




In [ ]:
plt.hist(data[:,2], bins=np.arange(0,101,1))

In [ ]:
# After filling the missing age fields with approxiamations
# i.e master/miss with 10
# mr/mrs with 10
a=pd.read_csv(r"/Users/vijay/Desktop/train_new.csv")
a['Sex'].replace(['male','female'],[1,2], inplace=True)
a['Age'].replace([np.NaN],[30], inplace=True)
a['SibSp'].replace([np.NaN],[0], inplace=True)
a['Parch'].replace([np.NaN],[0], inplace=True)
a['Fare'].replace([np.NaN],[0], inplace=True)
a['Pclass'].replace([np.NaN],[2], inplace=True)
a['Embarked'].replace(['S','C','Q'],[1,2,3], inplace=True)
a['Embarked'].replace([np.NaN],[2], inplace=True)
a['Survived'].replace([np.NaN],[1], inplace=True)
a

In [ ]:
y=np.array(a)[:,1]
y=y.astype('int')
a_reduced=a[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
a_reduced

In [ ]:
data=np.array(a_reduced)
data

In [ ]:
age_binned=np.digitize(data[:,2],bins=np.array([0,12,18,35,50,70,100]))
fare_binned=np.digitize(data[:,5],bins=np.array([0,15,40,70,100,200,300,400,550]))
data_binned=np.hstack([data[:,0:2],age_binned.reshape(-1,1),data[:,3:5],fare_binned.reshape(-1,1),data[:,6].reshape(-1,1)])

In [ ]:
X_train, X_test, y_train, y_test = tts(data_binned, y, random_state=20,stratify=y)

In [ ]:
from sklearn.svm import SVC
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                  'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
svc=SVC()
grid_search = GridSearchCV(svc, param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.score(X_test, y_test))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))


In [ ]:
svc=SVC(C=1, gamma=0.1)
svc.fit(X_train, y_train)

In [ ]:
#  gradient_boosting classifier
X_train, X_test, y_train, y_test = tts(data, y, random_state=20,stratify=y)
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {'max_depth': [2,3,4],
              'learning_rate': [0.1,0.2],
              'n_estimators': [ 50, 100, 150],
              'min_samples_split': [3,4,5],
              'min_samples_leaf': [1,2],
              'max_features': [1,2]}
gbc=GradientBoostingClassifier(random_state=12)
grid_search = GridSearchCV(gbc, param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.score(X_test, y_test))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
gbc=GradientBoostingClassifier(max_depth=4, max_features=2, min_samples_split=4,
                           n_estimators=50)

gbc.fit(X_train, y_train)



In [ ]:
# modified test data 
a1=pd.read_csv(r"/Users/vijay/Desktop/test_new.csv")
a1['Sex'].replace(['male','female'],[1,2], inplace=True)
a1['Age'].replace([np.NaN],[30], inplace=True)
a1['SibSp'].replace([np.NaN],[0], inplace=True)
a1['Parch'].replace([np.NaN],[0], inplace=True)
a1['Fare'].replace([np.NaN],[0], inplace=True)
a1['Pclass'].replace([np.NaN],[2], inplace=True)
a1['Embarked'].replace(['S','C','Q'],[1,2,3], inplace=True)
a1['Embarked'].replace([np.NaN],[2], inplace=True)
a1

In [ ]:
a1_reduced=a1[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
a1_reduced_data=np.array(a1_reduced)
a1_reduced_data

In [ ]:
prediction=gbc.predict(a1_reduced_data)
result=np.hstack([PassengerId,prediction.reshape(-1,1)])
dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
df.to_csv(r"/Users/vijay/Desktop/result.csv",index=False)

In [ ]:
age_binned=np.digitize(a1_reduced_data[:,2],bins=np.array([0,12,18,35,50,70,100]))
fare_binned=np.digitize(a1_reduced_data[:,5],bins=np.array([0,15,40,70,100,200,300,400,550]))
data_binned_a1=np.hstack([a1_reduced_data[:,0:2],age_binned.reshape(-1,1),a1_reduced_data[:,3:5],fare_binned.reshape(-1,1),a1_reduced_data[:,6].reshape(-1,1)])


In [ ]:
prediction=svc.predict(data_binned_a1)
result=np.hstack([PassengerId,prediction.reshape(-1,1)])
dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
df.to_csv(r"/Users/vijay/Desktop/result.csv",index=False)

In [ ]:
X_train, X_test, y_train, y_test = tts(data, y, random_state=20,stratify=y)
gbc=GradientBoostingClassifier(max_depth=3, max_features=2, min_samples_split=4,n_estimators=50,random_state=12)
gbc.fit(X_train, y_train)
print(gbc.score(X_train, y_train))
print(gbc.score(X_test, y_test))

In [ ]:
prediction=gbc.predict(a1_reduced_data)
result=np.hstack([PassengerId,prediction.reshape(-1,1)])
dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
df.to_csv(r"/Users/vijay/Desktop/result.csv",index=False)

In [ ]:
# If you change or don't change age columns with modified age, still it's same.
# After grid search also, it's not improving
# instead some manual tuning gives good score on Kaggle
# what about random forest?

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(max_depth=5,random_state=15)
rfc.fit(X_train, y_train)
print(rfc.score(X_train, y_train))
print(rfc.score(X_test, y_test))


In [ ]:
prediction=rfc.predict(a1_reduced_data)
result=np.hstack([PassengerId,prediction.reshape(-1,1)])
dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
df.to_csv(r"/Users/vijay/Desktop/result.csv",index=False)

In [ ]:
rfc.feature_importances_

In [ ]:
# let's try with only important features

In [ ]:
a_reduced

In [ ]:
a_reduced=a[['Pclass','Sex','Age','Fare']]
data=np.array(a_reduced)

In [ ]:
data=np.array(a_reduced)
X_train, X_test, y_train, y_test = tts(data, y, random_state=20,stratify=y)
rfc=RandomForestClassifier(max_depth=3,random_state=15)
rfc.fit(X_train, y_train)
print(rfc.score(X_train, y_train))
print(rfc.score(X_test, y_test))

In [ ]:
rfc.feature_importances_

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV 
param_grid = {'max_depth': [2,3,4],
              'learning_rate': [0.1,0.2],
              'n_estimators': [ 50, 100, 150],
              'min_samples_split': [3,4,5],
              'min_samples_leaf': [1,2],
              'max_features': [1,2]
gbc=GradientBoostingClassifier(random_state=12)
grid_search = GridSearchCV(gbc, param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.score(X_test, y_test))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
# start from here

In [ ]:
a=pd.read_csv(r"/Users/vijay/Downloads/train_new.csv")
a['Sex'].replace(['male','female'],[1,2], inplace=True)
a['Age'].replace([np.NaN],[30], inplace=True)
a['SibSp'].replace([np.NaN],[0], inplace=True)
a['Parch'].replace([np.NaN],[0], inplace=True)
a['Fare'].replace([np.NaN],[8], inplace=True)
a['Pclass'].replace([np.NaN],[2], inplace=True)
a['Embarked'].replace(['S','C','Q'],[1,2,3], inplace=True)
a['Embarked'].replace([np.NaN],[2], inplace=True)
a

In [ ]:
x = a['Name'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
x

In [ ]:
np.array(x)

In [ ]:
a_reduced=a[['Pclass','Sex','Fare','Age']]
data=np.array(a_reduced)

In [ ]:
data_final=np.hstack([data,np.array(x).reshape(-1,1)])


In [ ]:
X_train, X_test, y_train, y_test = tts(data_final, y, random_state=20,stratify=y)
rfc=RandomForestClassifier(max_depth= 4)
rfc.fit(X_train, y_train)
print(rfc.score(X_train, y_train))
print(rfc.score(X_test, y_test))

In [ ]:
rfc.feature_importances_

In [ ]:
gbc=GradientBoostingClassifier(max_depth=2,random_state=12)
gbc.fit(X_train, y_train)
print(gbc.score(X_train, y_train))
print(gbc.score(X_test, y_test))

In [ ]:
a1=pd.read_csv(r"/Users/vijay/Downloads/test_new.csv")
a1['Sex'].replace(['male','female'],[1,2], inplace=True)
a1['Age'].replace([np.NaN],[30], inplace=True)
a1['SibSp'].replace([np.NaN],[0], inplace=True)
a1['Parch'].replace([np.NaN],[0], inplace=True)
a1['Fare'].replace([np.NaN],[8], inplace=True)
a1['Pclass'].replace([np.NaN],[2], inplace=True)
a1['Embarked'].replace(['S','C','Q'],[1,2,3], inplace=True)
a1['Embarked'].replace([np.NaN],[2], inplace=True)
a


In [ ]:
x = a1['Name'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
x

In [ ]:
np.array(x)

In [ ]:
a1_reduced=a1[['Pclass','Sex','Fare','Age']]
data=np.array(a1_reduced)


In [ ]:
data_final=np.hstack([data,np.array(x).reshape(-1,1)])


In [ ]:
data_final_a1=np.hstack([np.array(a1_reduced),np.array(x).reshape(-1,1)])
data_final_a1.shape


In [ ]:
prediction=rfc.predict(data_final_a1)
result=np.hstack([PassengerId,prediction.reshape(-1,1)])
dict = {'PassengerId': result[:,0], 'Survived': result[:,1]} 
df = pd.DataFrame(dict,index=None)
df.to_csv(r"/Users/vijay/Desktop/result_prefix.csv",index=False)


In [ ]:
# End project here
# Title is more inportant feature than sex
# probably because it contains information like age also
#  But even with this feature accuracy doesnot increase beyond maximum achieved
